In [ ]:
import tarfile
import shutil
import os
import cv2
import pathlib
import numpy as np
import pandas as pd
import gc
gc.enable()

import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.resnet import ResNet101

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

# Importazione dataset

In [ ]:
%%capture
drive.mount('/content/gdrive', force_remount=True)
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto_VIPM/dataset/new_train_set.tar'
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto_VIPM/dataset/val.tar'
!unzip '/content/gdrive/MyDrive/Progetto_VIPM/dataset/val_degraded.zip' -d '/content/'

## Train

In [ ]:
%%capture
csv_train_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/new_train_info.csv", dtype=str)
csv_train_file.columns = ['filename', 'label']
parent_dir = "new_train_set/"
labels = csv_train_file['label']
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_train_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    shutil.copy(path, new_path)

In [ ]:
train_data_dir = pathlib.Path('new_train_set/')
seed_train_validation = 42 # must be same for train_ds and val_ds
shuffle_value = True
validation_split = 0.2
batch_size = 512

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="training",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="validation",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 250999 files belonging to 251 classes.
Using 200800 files for training.
Found 250999 files belonging to 251 classes.
Using 50199 files for validation.


## Validation

In [ ]:
%%capture
csv_test_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_file.columns = ['filename', 'label']
parent_dir = "val_set/"
labels = csv_test_file['label']
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

In [ ]:
%%capture
csv_test_degraded_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_degraded_file.columns = ['filename', 'label']
parent_dir = "val_set_degraded/"
labels = csv_test_degraded_file['label']
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

In [ ]:
test_data_dir = pathlib.Path('val_set/')
test_degraded_data_dir = pathlib.Path('val_set_degraded/')

shuffle_value = True

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

test_degraded_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_degraded_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 11993 files belonging to 251 classes.
Found 11993 files belonging to 251 classes.


# Creazione modello

In [ ]:
gc.collect()
base_eff_model = EfficientNetB0(weights="imagenet", include_top=True, input_shape=(224,224,3))
base_eff_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_2 (Rescaling)        (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 normalization_2 (Normalization  (None, 224, 224, 3)  7          ['rescaling_2[0][0]']            
 )                                                                                                
                                                                                     

In [ ]:
trainable_layer = False
for layer in base_eff_model.layers:
    if layer.name == "block7a_expand_conv":
        trainable_layer=True
    layer.trainable=trainable_layer

eff_model = Model(inputs=base_eff_model.input, outputs=base_eff_model.get_layer('top_dropout').output)
last_layer = base_eff_model.get_layer('top_dropout').output
flatten_layer = Flatten()(last_layer)
classification_layer = Dense(251, activation='softmax')(flatten_layer)
eff_model = Model(inputs=eff_model.input, outputs=classification_layer)
# eff_model.summary()

In [ ]:
eff_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
history = eff_model.fit(train_ds, epochs=15, verbose=1, validation_data=val_ds)
eff_model.save('/content/gdrive/MyDrive/Progetto_VIPM/models/effNetB0_augmented.h5')

In [ ]:
eff_model = keras.models.load_model('/content/gdrive/MyDrive/Progetto_VIPM/models/effNetB0_augmented.h5')

eff_model.evaluate(test_ds, batch_size=512)
eff_model.evaluate(test_degraded_ds, batch_size=512)

24/24 [==============================] - 36s 1s/step - loss: 2.9612 - accuracy: 0.4035 - top_k_categorical_accuracy: 0.5907


[2.9612293243408203, 0.40348535776138306, 0.5906779170036316]